Connect to vCenter Server
===

- Connect to vCenter
- Load the content

In [37]:
import config
import vsphere_client as vsphere

# vc-39
# vc-169 - with a 5.5 ESXi host
# vc-drteam
conf = config.load_yml_conf(
    '/Users/weijin.wang/ws/.secrets/appflows.yml', 'vc-39')

vcc = vsphere.VSphereClient(
    hostname=conf['url'],
    username=conf['username'],
    password=conf['password']
)

_vms = vcc.list_vms()
print('Find {} vms'.format(len(_vms)))

_datastores = vcc.list_objects_by_type(vsphere.VIM_DATA_STORE)
print('Find {} datastores'.format(len(_datastores)))
_accessible_datastores = [ds for ds in _datastores if ds.summary.accessible]
print('\t{} of them are accessible'.format(len(_accessible_datastores)))

_hosts = vcc.list_objects_by_type(vsphere.VIM_HOST)
print('Find {} hosts:'.format(len(_hosts)))
for host in _hosts:
    print('\t{}'.format(host.name))



Find 83 vms
Find 5 datastores
	4 of them are accessible
Find 1 hosts:
	dr_cluster


In [21]:

vms = vcc.list_vms()
print('find {} vms'.format(len(vms)))
for vm in vms:
    print(vm.name)


find 88 vms
Rubrik_Managed_autotest-prem--20200310-114022-121564 03-10 11:54
Rubrik_Managed_april-06 03-14 23:47
Rubrik_Managed_autotest-prem--20200306-110653-471360 03-12 10:49
april-07
Rubrik_Managed_autotest-prem--20200312-155133-771679 03-12 16:05
Rubrik_Managed_april-06 02-28 14:49


april-static-03
Centos6-7-2disk
april-win-01
april-onprem-10
Deprecated demo-vm-03 02-29 04:05
autotest-prem--20200310-111910-418623-test 03-13 17:10


Deprecated april-06 02-28 22:28
Deprecated april-03 02-27 23:11
rohit-centos-6-10-2disk
autotest-prem--20200305-173708-385643
april-static-04
Rubrik_Managed_april-onprem-02 02-14 14:32 02-14 16:52
april-onprem-04
april-onprem-09
april-onprem-06


april-cent7-nocloudinit
Deprecated april-onprem-02 02-27 22:59
hilary-2-test-vm
win_server_2012r2_20gb_4mem_1cpu
rohit-rhel-6-10-2disks
april-03
Deprecated demo-vm-05 03-02 19:44
Rubrik_Managed_april-onprem-02 02-20 11:51 02-22 21:03
Centos-7-0-template
demo-vm-01
april-onprem-02
april-static-01


Rubrik_Managed_april-12 03-15 00:29
Rubrik_Managed_autotest-prem--20200312-121555-219005 03-12 12:25
april-onprem-05
demo-vm-03
rohit-rhel-6-9
Deprecated april-04 03-02 23:25
metadata_ubuntu_vm1
ubuntu_base_template
demo-vm-04
template-centos7-onpremscript


april-onprem-01
demo-vm-02
april-static-02
centos7-onprem-template
Rubrik_Managed_autotest-prem--20200314-161727-975678 03-14 16:33
ubuntu_4gb_template
april-onprem-11


autotest-prem--20200312-190041-728495
april-06-test 03-04 11:07
april-08-test 03-11 03:11
Rubrik_Managed_autotest-prem--20200306-121741-816320 03-07 15:30
Rubrik_Managed_april-05 02-27 15:47
autotest-prem--20200305-113701-524545
Rubrik_Managed_koki-static-test 03-03 16:41
april-02
Rubrik_Managed_april-08 03-10 17:22
CdmToCdmRohitCentos6_10


Rubrik_Managed_april-06 03-09 23:50
test export april prod05
april-onprem-08
curious george
Rubrik_Managed_april-05 03-09 23:49
Rubrik_Managed_april-onprem-03 02-14 14:22 02-14 14:55
Deprecated april-04 03-03 01:08
Rubrik_Managed_autotest-prem--20200305-113701-524545 03-05 19:45
_weijin_window_vm
april-onprem-07


centos7-prem-template
Deprecated _weijin_0301 03-03 21:38
centos7-tool-template
Rubrik_Managed_april-static-01 03-07 19:44
Rubrik_Managed_april-05 02-27 16:07
Deprecated _weijin_03 02-23 05:36
CdmToCdmRohitRhel6_7
Rubrik_Managed_april-05 02-27 15:47*
Deprecated demo-vm-04 02-29 04:05


Rubrik_Managed_autotest-prem--20200310-111910-418623 03-10 11:45
Deprecated demo-vm-06 03-02 19:44
centos7-postscript-template
rohit_rhel_6_7
april-onprem-03
april-01
Deprecated april-01 02-27 22:30
hilary-test-vm
curious george*


In [6]:

vm = vcc.find_by_name(vsphere.VIM_VIRTUAL_MACHINE, 'april-onprem-04')

template = vm
if template:
    print('Template found: {}'.format(template.name))
else:
    print("No template found")


Template found: april-onprem-04


In [11]:
for i in range(1):
    vcc.clone_vm(template, "_weijin_0310_{}".format(i), ds_name='localstore')


Cloning VM to ...
	 Data Center  : 'Colo-DC1'
	 VM Folder    : 'vm'
	 Resource Pool: 'Resources'
	 Data store   : 'localstore'


VM '_weijin_0310_0' cloned!


In [47]:
vms = vcc.list_vms("Deprecated")
for vm in vms:
    print(vm.name, vm.power_state)


Deprecated prem-p0-20200408-230940-340952 04-09 07:04 poweredOff
Deprecated prem-p0-20200407-141237-890995 04-08 23:21 poweredOff
Deprecated prem-p0-20200408-230459-996360 04-09 07:04 poweredOff
Deprecated demo-vm-07 04-08 23:44 poweredOff


In [48]:
for vm in vms:
    if vm.power_state != 'poweredOff':
        print("Powering off VM: %s" % vm.name)
        task = vm.data.PowerOffVM_Task()
        vcc.wait_for_task(task)
        print("task status: {}".format(task.info.state))
        
    print("Destroying VM: %s" % vm.name)
    task = vm.data.Destroy_Task()
    vcc.wait_for_task(task)
    print("task status: {}".format(task.info.state))



Destroying VM: Deprecated prem-p0-20200408-230940-340952 04-09 07:04


task status: success
Destroying VM: Deprecated prem-p0-20200407-141237-890995 04-08 23:21


task status: success
Destroying VM: Deprecated prem-p0-20200408-230459-996360 04-09 07:04


task status: success
Destroying VM: Deprecated demo-vm-07 04-08 23:44


task status: success


In [16]:
datastores = [ds for ds in vcc.list_datastores('rubrik_') if not ds.accessible]
for ds in datastores:
    print(ds.name)



In [7]:
print('Deleting inaccessible dastastores with prefix \"rubrik_\" ...')
for ds in datastores:
    ds_name = ds.name
    for h in ds.hosts:
        for d in h.datastore:
            if d._moId == ds.moid:
                h.configManager.datastoreSystem.RemoveDatastore(ds.data)
                print("Removed datastore {} from host {}".format(
                    ds_name, h.name
                ))





Deleting inaccessible dastastores with prefix "rubrik_" ...


Removed datastore rubrik_bb95d8b1e56947aeaa411d85e78c1a07 from host esx-5-216.colo.rubrik.com


Removed datastore rubrik_f8e14e75586b4141bccde387708e4937 from host esx-5-216.colo.rubrik.com


Removed datastore rubrik_0dfa72afe7634606a6559dffac85bb1f from host esx-5-216.colo.rubrik.com
